In [1]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(''))))

from src.pipeline import Pipeline
from src.data.preprocessing import removal, tokenization, stemming

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jonas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jonas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pipeline = Pipeline()

In [3]:
pipeline.setup()

In [4]:
pipeline.preprocess()

100%|██████████| 200/200 [00:00<00:00, 849.77it/s]


In [5]:
pipeline.create_w2v_embeddings()

0 Unknown words replaced with zero vecs

329 Unknown words replaced with zero vecs

5 Unknown words replaced with zero vecs



In [6]:
pipeline.create_w2v_feature()

 76%|███████▋  | 3910714/5128266 [8:28:18<2:38:15, 128.23it/s]    


KeyboardInterrupt: 

In [ ]:
embeddings = load('data/embeddings/tfidf_collection_embeddings.pkl')
embeddings_queries = load('data/embeddings/tfidf_query_embeddings.pkl')

embeddings_queries[pipeline.queries[pipeline.queries['qID'] == 914462].index]
#embeddings[pipeline.collection[pipeline.collection['pID'] == 3852073].index]

In [ ]:
from src.features.generator import create_all
create_all(features_test, pipeline.collection, pipeline.queries_test)

In [ ]:
pipeline.queries

In [7]:
pipeline.collection

,pID,Passage,preprocessed,w2v
0,945864,Notice how their voice goes up and down very d...,"[notic, voic, goe, deliber, know, well, notic,...","[-10.653138, 13.532735, 7.286613, -11.573866, ..."
1,2776470,Definition of inter-. 1 1 : between : among :...,"[definit, inter, 1, 1, among, midst, intercrop...","[-2.2495942, 8.763295, 5.63466, 16.018324, -26..."
2,1897608,"Literary consonance. From Wikipedia, the free ...","[literari, conson, wikipedia, free, encycloped...","[-13.868881, 86.55683, 50.701267, 21.939404, -..."
3,2707038,"ABOUT MARINELLO. For more than 100 years, Mari...","[marinello, 100, year, marinello, educ, studen...","[-2.7350445, 15.263592, -7.922319, 7.6605225, ..."
4,5712720,3. Ginger. Due to its anti-inflammatory proper...,"[3, ginger, due, antiinflammatori, properti, g...","[-95.00605, 96.29297, 67.63398, 2.271606, -29...."
...,...,...,...,...
999995,7604415,[8] The Regulations specifically define a 'sur...,"[8, regul, specif, defin, surrog, speci, recip...","[-5.0366545, 13.667796, -4.856635, -8.303628, ..."
999996,4995189,1 Animals and plants get rid of carbon dioxide...,"[1, anim, plant, get, rid, carbon, dioxid, ga,...","[-112.50996, -5.302109, -48.799213, 44.034668,..."
999997,6668055,Here;s how: 1. Reduces muscle tension[1]: A si...,"[1, reduc, muscl, tension, 1, simpl, warm, bat...","[-82.84231, 66.0176, 31.196491, -32.56941, -19..."
999998,142244,dignified (dÉªgnÉªfaÉªd ) adjective. If you sa...,"[dignifi, déªgnéªfaéªd, adject, say, someon, s...","[12.142402, 18.530884, 7.612073, 0.72554255, 0..."


In [ ]:
pipeline.qrels

In [ ]:
pipeline.preprocess()

In [ ]:
pipeline.create_BM25_features()

In [ ]:
pipeline.collection

In [ ]:
pipeline.create_w2v_embeddings()

In [ ]:
pipeline.create_tfidf_embeddings()

In [ ]:
pipeline.create_tfidf_feature()

In [ ]:
pipeline.create_jaccard_feature()

In [ ]:
pipeline.create_sentence_features()

In [ ]:
pipeline.create_interpretation_features()

In [ ]:
pipeline.create_POS_features()

In [ ]:
pipeline.queries_test

In [8]:
pipeline.features

,qID,pID,y
0,1003840,4063719,1
114,100386,6182295,1
223,1024573,4338653,1
344,1031605,4415474,1
454,1040949,4955943,1
...,...,...,...
5106811,997626,6217581,0
5106812,997626,2725359,0
5106813,997626,6142664,0
5106814,997626,4900395,0


In [ ]:
min(pipeline.features['w2v_cosine'])

In [ ]:
import pandas as pd
import numpy as np

pipeline.collection.preprocessed.apply(lambda passage: np.isin('wikipedia', passage)).value_counts()[1]

In [ ]:
pipeline.collection.preprocessed.size

In [ ]:
len(pipeline.collection.preprocessed[2])

In [ ]:
pipeline.collection.preprocessed.progress_apply(lambda passage: passage.size).mean()

In [ ]:
from src.features.generator import create_tfidf_embeddings, create_all
from src.utils.utils import load 

create_all(features_test, pipeline.collection, pipeline.queries_test, load('models/tfidf.pkl'))

In [ ]:
create_all(features_test, pipeline.collection, pipeline.queries_test)

In [ ]:
pipeline.queries_test

In [ ]:
import pandas as pd
features_test = pd.DataFrame()
for index, query in pipeline.queries_test.iterrows():
    features_test = pd.concat([features_test, pd.DataFrame({
        'qID': [query['qID']] * len(pipeline.collection),
        'pID': pipeline.collection['pID']
    })])